In [7]:
import pandas as pd
# Configurar para exibir todas as colunas
pd.set_option('display.max_columns', None)
import numpy as np
#import sqlite3

import os
import glob
import warnings
warnings.filterwarnings("ignore")

In [8]:
ligas = {'England' : [39],
        'Spain' : [140],
        'France': [61],
        'Brazil' : [71],
        'Germany' : [78],
        'Italy' : [135],
        'Netherlands' : [88],
        'Argentina' : [128]}

In [9]:
def get_data_liga(liga):

    caminho_diretorio = r'data\processed\batch_article'

    arquivos_xlsx = glob.glob(os.path.join(caminho_diretorio, '*.xlsx'))

    arquivos = [arquivo for arquivo in arquivos_xlsx if f'matchs_{liga}_' in arquivo]

    df = pd.read_excel(arquivos[0])

    df["Kick Off"] = pd.to_datetime(df['Kick Off'])

    df = df.sort_values(by='ts')

    df.reset_index().drop(columns={'index'}, inplace=True)

    return df

def get_number_teams(df):

    number_teams = pd.concat([df['teams_home_id'], df['teams_away_id']]).unique()

    return len(number_teams)

def get_number_players(df):
    
    number_players = pd.Series(df.filter(like='player_').stack().values).unique()
    
    return len(number_players)

def get_percentage_results(df):

    df['result'] = '0'
    df['result'] = np.where(df['Home Score'] > df['Away Score'], '1', df['result'])
    df['result'] = np.where(df['Home Score'] < df['Away Score'], '2', df['result'])

    return (df['result'].value_counts() / len(df)).to_dict()

def get_years_league(df):
 
    first = df['league_season'].sort_values().unique()[0]
    last = df['league_season'].sort_values().unique()[-1]

    return first, last

In [4]:
lig = []
number_games = []
number_teams = []
number_players = []
percentage_home = []
percentage_draw = []
percentage_away = []
first_year = []
last_year = []

for liga in ligas.keys():
    
    df = get_data_liga(ligas[liga][0])

    number_games.append((len(df)))

    lig.append(liga)

    number_teams.append(get_number_teams(df))

    number_players.append(get_number_players(df))

    percentage_results = get_percentage_results(df)

    percentage_home.append(round(percentage_results['1']*100, 2))

    percentage_draw.append(round(percentage_results['0']*100, 2))

    percentage_away.append(round(percentage_results['2']*100, 2))

    years = get_years_league(df)

    first_year.append(years[0])

    last_year.append(years[1])

In [5]:
pd.DataFrame({'liga' : lig,
              'number_games' : number_games,
              'number_teams' : number_teams,
              'number_players' : number_players,
              'percentage_home' : percentage_home,
              'percentage_draw' : percentage_draw,
              'percentage_away' : percentage_away,
              'first_year' : first_year,
              'last_year' : last_year})

,liga,number_games,number_teams,number_players,percentage_home,percentage_draw,percentage_away,first_year,last_year
0,England,2932,32,1381,44.47,23.53,31.99,2014,2022
1,Spain,2918,30,1523,45.41,26.18,28.41,2015,2022
2,France,2842,31,1662,43.46,26.39,30.15,2015,2022
3,Brazil,2809,33,2034,47.92,27.55,24.53,2015,2022
4,Germany,2374,29,1253,44.82,24.43,30.75,2015,2022
5,Italy,2929,33,1480,43.15,24.38,32.47,2015,2022
6,Netherlands,2345,28,1536,45.63,23.03,31.34,2015,2022
7,Argentina,2607,37,2130,42.88,28.46,28.65,2016,2023


In [49]:
lig = []
number_games = []
number_teams = []
number_players = []
percentage_home = []
percentage_draw = []
percentage_away = []
first_year = []
last_year = []

dfs = []

for liga in ligas.keys():
    
    df = get_data_liga(ligas[liga][0])

    df['Result'] = np.where(df['Home Score'] > df['Away Score'], 1, np.where(df['Home Score'] < df['Away Score'], 2, 0))

    df = df.groupby(['league_season', 'Country'])['Result'].value_counts().unstack(fill_value=0).reset_index()

    df.columns = ['league_season','Country', 'Draw', 'Home', 'Away']

    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df = df.groupby(['league_season']).agg({
                                        'Draw' : 'sum',
                                        'Home' : 'sum',
                                        "Away" : 'sum'
                                    }).reset_index()

df['Total'] = df['Draw'] + df['Home'] + df['Away']

df['Draw'] = df['Draw'] / df['Total']

df['Away'] = df['Away'] / df['Total']

df['Home'] = df['Home'] / df['Total']

df.to_excel('distribuicao_d_h_a_por_ano.xlsx')

,league_season,Draw,Home,Away,Total
0,2014,0.333333,0.333333,0.333333,3
1,2015,0.255945,0.448768,0.295288,2313
2,2016,0.235664,0.484965,0.279371,2860
3,2017,0.252078,0.449446,0.298476,2888
4,2018,0.262249,0.463518,0.274233,2837
5,2019,0.247391,0.452648,0.299961,2587
6,2020,0.256895,0.404609,0.338496,2647
7,2021,0.263123,0.433555,0.303322,3010
8,2022,0.267707,0.437775,0.294518,2499
9,2023,0.303571,0.410714,0.285714,112
